EDA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import yfinance as yf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "SPY",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "5d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )
data.reset_index(inplace=True,drop=False)
print(data)

[*********************100%***********************]  1 of 1 completed
                      Datetime        Open        High         Low  \
0    2020-11-18 09:30:00-05:00  360.760010  361.010010  360.709991   
1    2020-11-18 09:31:00-05:00  360.739990  360.760010  360.579987   
2    2020-11-18 09:32:00-05:00  360.679993  360.820007  360.579987   
3    2020-11-18 09:33:00-05:00  360.720001  360.730011  360.559998   
4    2020-11-18 09:34:00-05:00  360.660004  360.760010  360.619995   
...                        ...         ...         ...         ...   
1945 2020-11-24 15:55:00-05:00  363.260010  363.399994  363.260010   
1946 2020-11-24 15:56:00-05:00  363.364990  363.369995  363.109985   
1947 2020-11-24 15:57:00-05:00  363.149994  363.359985  363.089996   
1948 2020-11-24 15:58:00-05:00  363.364990  363.369995  362.980011   
1949 2020-11-24 15:59:00-05:00  363.190002  363.309998  363.070007   

           Close   Volume  
0     360.760010  1539385  
1     360.679993   230986  
2     

ETL

In [3]:
import pymongo
client = pymongo.MongoClient()

In [4]:
db = client.get_database("stock")
collection = db.get_collection("stock")
update_count = 0
for record in data.to_dict('records'):
    result = collection.replace_one(
        filter={'Datetime': record['Datetime']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print(f"rows={data.shape[0]}, update={update_count}, "
      f"insert={data.shape[0]-update_count}")

rows=1950, update=0, insert=1950
